# Import Library

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import glob # to find files


import seaborn as sns


from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers

# import VGG19 to perform transfer learning
from tensorflow.keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input

# Creating Dataset:
<br> We want to grab jpg files from folder and create a dataset </br>
<br> training_folder : containing all training chest images of normal/pneumonia </br>
<br> testing_folder : containing all testing chest images of normal/pneumonia </br>
<br> validation_folder : containing all validation chest images of normal/pneumonia for purpose of validation </br>

In [3]:
path = '../input/chest-xray-pneumonia/chest_xray/'

# train directory
train_folder=path+"train/"
train_normal_dir=train_folder+"NORMAL/"
train_pneu_dir=train_folder+"PNEUMONIA/"

# test directory
test_folder=path+"test/"
test_normal_dir=test_folder+"NORMAL/"
test_pneu_dir=test_folder+"PNEUMONIA/"

# validation directory
val_folder=path+"val/"
val_normal_dir=val_folder+"NORMAL/"
val_pneu_dir=val_folder+"PNEUMONIA/"

# variables for image size (VGG19 uses 224,224 as input, we should change size of image to 224)
img_width=224
img_height=224

# variable for model
batch_size=64
epochs=10

## Create list of classes

In [4]:
# Train Dataset
train_class_names=os.listdir(train_folder)
print("Train class names: %s" % (train_class_names))
# print("\n")

# Test Dataset
test_class_names=os.listdir(test_folder)
print("Test class names: %s" % (test_class_names))
# print("\n")

# Validation Dataset
val_class_names=os.listdir(val_folder)
print("Validation class names: %s" % (val_class_names))

## Find all jpeg and put in list 

In [5]:
# find all files, our files has extension jpeg
train_normal_cases = glob.glob(train_normal_dir + '*jpeg')
train_pneu_cases = glob.glob(train_pneu_dir + '*jpeg')

test_normal_cases = glob.glob(test_normal_dir + '*jpeg')
test_pneu_cases = glob.glob(test_pneu_dir + '*jpeg')

val_normal_cases = glob.glob(val_normal_dir + '*jpeg')
val_pneu_cases = glob.glob(val_pneu_dir + '*jpeg')

# create lists for train, test & validation cases, create labels as well
train_list = []
test_list = []
val_list = []

for x in train_normal_cases:
    train_list.append([x, "Normal"])
    
for x in train_pneu_cases:
    train_list.append([x, "Pneumonia"])
    
for x in test_normal_cases:
    test_list.append([x, "Normal"])
    
for x in test_pneu_cases:
    test_list.append([x, "Pneumonia"])
    
for x in val_normal_cases:
    val_list.append([x, "Normal"])
    
for x in val_pneu_cases:
    val_list.append([x, "Pneumonia"])

# create dataframes for training, test, valid
train_df = pd.DataFrame(train_list, columns=['image', 'Diagnos'])
print(train_df.shape)
test_df = pd.DataFrame(test_list, columns=['image', 'Diagnos'])
print(test_df.shape)
val_df = pd.DataFrame(val_list, columns=['image', 'Diagnos'])
print(val_df.shape)

In [6]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
print(train_df)


In [7]:
train_df['Diagnos'].value_counts()

#### Because our dataset has imbalance number of Normal/Pneumonia training examples, our model might tend to predict Penumonia images more accurately

# Change jpeg to array 
Change jpeg into array so we can use numpy / tensorflow on them. or plot them.

In [8]:
# Declaring variables
x=[] # to store array value of the images
y=[] # to store the labels of the images

for folder in os.listdir(train_folder):
    image_list=os.listdir(train_folder+"/"+folder)
    for img_name in image_list:
        # Loading images
        img=image.load_img(train_folder+"/"+folder+"/"+img_name,target_size=(img_width,img_height))
        
        # Converting to arrary
        img=image.img_to_array(img)
        
        # Transfer Learning: this is to apply preprocess of VGG19 model to our images before passing it to VGG19
        img=preprocess_input(img) # Required since we using transfer-learning 
        
        # Appending the arrarys
        x.append(img) # appending image array
        y.append(train_class_names.index(folder)) # appending class index to the array
        
print("Training Dataset")

In [9]:
def showimg(x, y):
    plt.figure(figsize=(10,10))
    
    # Plotting 25 images
    for n in range(25):
        
        
        ax = plt.subplot(5,5,n+1)
       
        plt.imshow(x[n])
        if y[n]:
            plt.title("PNEUMONIA")
        else:
            plt.title("NORMAL")
        plt.axis("off")

showimg(x,y)

In [10]:
# Declaring variables
val_images=[]
val_images_Original=[]
val_image_label=[] # to store the labels of the images

for folder in os.listdir(val_folder):
    image_list=os.listdir(val_folder+"/"+folder)
    for img_name in image_list:
        # Loading images
        img=image.load_img(val_folder+"/"+folder+"/"+img_name,target_size=(img_width,img_height))
        
        # Converting to arrarys
        img=image.img_to_array(img)
        
        # Saving original images, will be used just for display at the end
        val_images_Original.append(img.copy())
        
        # Transfer Learning: this is to apply preprocess of VGG19 to our images before passing it to VGG19
        img=preprocess_input(img) #  Optional step
        
        # Appending arrays
        val_images.append(img) # appending image array
        val_image_label.append(val_class_names.index(folder))
        
print("Validation Dataset")

In [11]:
# Declaring variables
test_images=[]
test_images_Original=[]
test_image_label=[] # to store the labels of the images

for folder in os.listdir(test_folder):
    image_list=os.listdir(test_folder+"/"+folder)
    for img_name in image_list:
        # Loading images
        img=image.load_img(test_folder+"/"+folder+"/"+img_name,target_size=(img_width,img_height))
        
        # Converting to arrarys
        img=image.img_to_array(img)
        
        # Saving original images, will be used just for display at the end
        test_images_Original.append(img.copy())
        
        # Transfer Learning: this is to apply preprocess of VGG19 to our images 
        img=preprocess_input(img) 
        
        # Appending arrays
        test_images.append(img) # appending image array
        test_image_label.append(test_class_names.index(folder))
        
print("Test Dataset")

In [12]:
# Training Dataset
print("Training Dataset:")

x=np.array(x) # Converting to np arrary to pass to the model
print(x.shape)

y=to_categorical(y) # onehot encoding of the labels (Sparse Matrix)
# print(y)
print(y.shape)

# ===========

# Test Dataset
print("Test Dataset:")

test_images=np.array(test_images) 
print(test_images.shape)

test_image_label=to_categorical(test_image_label) # onehot encoding of the labels)
print(test_image_label.shape)

# ===========

# Validation Dataset
print("Validation Dataset:")

val_images=np.array(val_images) 
print(val_images.shape)

val_image_label=to_categorical(val_image_label) # onehot encoding of the labels)
print(val_image_label.shape)

In [13]:
print(y)

# Deep Learning Model

In [14]:
from tensorflow.keras.applications import VGG19
model = VGG19(weights='imagenet')
model.summary()


## Input Image to VGG19 must be (224, 224, 3) where 3 represents color channel

In [15]:
input_layer=layers.Input(shape=(img_width, img_height, 3)) 
model=VGG19(weights='imagenet',input_tensor=input_layer,include_top=False)
model.summary()

We flatten the last layer and change output to 2 nodes for softmax function because we have 2 outcomes, <br> Normal chest / Not Normal chest </br>

In [16]:
last_layer=model.output
flatten= layers.Flatten()(last_layer)
output_layer=layers.Dense(2,activation='softmax')(flatten)
model=models.Model(inputs=input_layer,outputs=output_layer)
model.summary()


In [17]:
print("We are making all the layers intrainable except the last layer. \n")
for layer in model.layers[:-1]:
    layer.trainable=False

#### 80% training data, 20% testing data

In [18]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=5)

#### Must use Categorical_crossentropy because we have 2 outputs and we use one-hot encoding on our y-values

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [20]:
history=model.fit(xtrain,ytrain,epochs=10,batch_size=batch_size,verbose=True,validation_data=(xtest,ytest))

In [21]:
accuracy = history.history['accuracy']
val_accuracy  = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


In [22]:
plt.figure(figsize=(15,10))

plt.subplot(2, 2, 1)
plt.plot(accuracy, label = "Training accuracy")
plt.plot(val_accuracy, label="Validation accuracy")
plt.ylim(0.8, 1)
plt.legend()
plt.title("Training vs validation accuracy")


plt.subplot(2,2,2)
plt.plot(loss, label = "Training loss")
plt.plot(val_loss, label="Validation loss")
plt.ylim(0, 0.5)
plt.legend()
plt.title("Training vs validation loss")

plt.show()

In [23]:
"""
def Get_Xray_Type(argument):
    switcher = {
        "NORMAL": "Normal",
        "PNEUMONIA": "Pneumonia",
    }
    return switcher.get(argument, "Invalid X-ray")
def predict(img_name):
    img=image.load_img(img_name,target_size=(img_width,img_height))
    img=image.img_to_array(img)
    plt.imshow(img.astype('int32'))
    plt.show()
    img=preprocess_input(img)
#     plt.imshow(img.astype('int32'))
#     plt.show()
    prediction=model.predict(img.reshape(1,img_width,img_height,3))
    output=np.argmax(prediction)
    print(train_class_names[output] + ": " + Get_Xray_Type(train_class_names[output]))
    """